# Main Feedback Generation Notebook
This notebook handles preprocessing, model interaction, and feedback generation.

**--Set up: Github, Paths, Imports, Installs**

In [1]:
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
# !git branch -a

# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout -b feature/generated_rubric origin/feature/generated_rubric

%pip install -r requirements.txt --quiet
%pip install --upgrade openai --quiet

/content
fatal: destination path 'project' already exists and is not an empty directory.
/content/project
fatal: A branch named 'feature/generated_rubric' already exists.


**--Default imports**

In [2]:
# Add src folder to python path to edit python files
import sys
sys.path.append('/content/project/')

from google.colab import userdata
import os
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key

**--Import modules (you’re working on)**
<br>*each of our classes will be what will merge to this notebook (I'm pretty sure)*
<br>only loads what you explicitly request
<br>(this helps keep memory low and import fast)
<br> *the following is an example with my Rubric module*

**--Lauren's bit that can run the feedback generator here to test**
- ended up modularizing code to prepare for gradio/UI use.
  - had all files include ability to use uploaded files, used local testing files as default to be able to test
  - created feedback_engine file to be able to create runnable program in notebook to test
  - TODO: create runnable feedback program for Gradio/UI when that time comes

In [3]:
# from tropos.preprocess_docx import StudentSubmission
# from utils.feedback_formatting import format_feedback_blocks
#
# # Provide paths to your example file and requirements file
# example_path = "/content/project/data/raw/Student_1/Student_1_Part_1.docx"
# requirements_path = "/content/project/data/raw/Requirements.docx"
#
# # Load the student submission
# example_submission = StudentSubmission(example_path, requirements_path)
#
# # Print all key parts for visual verification
# print("📄 Submission Text:\n", example_submission.get_submission_text()[:1000])
# print("\n🗒️ Comments Text:\n", example_submission.get_comments_text())
# print("\n📌 Assignment Requirements:\n", example_submission.get_requirements_text()[:1000])
# print("\n📋 Rubric:\n", example_submission.get_rubric_prompt())
#

In [4]:
from tropos.models.prompt_builder import build_prompt

# Let's say you're testing FewShot (you can switch to ZeroShot or OneShot too)
prompt_type = "FewShot"

# # Simulate multiple examples (can be just one for testing)
# example_submissions = [example_submission]  # reuse same one for now
#
# # Build the actual prompt
# prompt = build_prompt(prompt_type, examples=example_submissions, target=example_submission)
#
# # Print the result to verify
# print("\n🧪 FINAL PROMPT OUTPUT:\n")
# print(prompt)  # trim if it's huge


In [5]:
from tropos import test_feedback_console

# test_feedback_console(
#     requirements_path="/content/project/data/raw/Requirements.docx",
#     example_dir="/content/project/data/raw",
#     target_dir="/content/project/data/unmarked_raw",
#     output_dir="/content/project/data/generated_output",
#     verbose=True
# )

In [6]:
# from utils.student_loader import load_all_student_examples_recursive
#
# example_dir = "/content/project/data/raw"
# requirements_path = "/content/project/data/raw/Requirements.docx"
#
# examples = load_all_student_examples_recursive(example_dir, requirements_path, verbose=True)
# print(len(examples), "examples loaded.")

In [7]:
# import zipfile
#
# with zipfile.ZipFile("/content/project/data/raw/Student_1/Student_1_Part_1.docx") as z:
#     print(z.namelist())
#

In [10]:
def explore_examples(requirements_path, example_dir):
    from utils.student_loader import load_all_student_examples_recursive
    from tropos.preprocess_docx.assignment_requirements import parse_requirements
    from docx import Document

    print("\n📌 Assignment Requirements:\n")
    parsed_req = parse_requirements(requirements_path)
    print(parsed_req.get_instructions())

    examples = load_all_student_examples_recursive(example_dir, requirements_path)

    print(f"\n📋 Rubric:\n{examples[0].rubric.format_clean_only()}\n")

    for i, example in enumerate(examples):
        print(f"\n🔍 Example {i+1}: {example.submission_path}")

        print("\n📝 Submission Text:\n")
        print(example.get_submission_text()[:1000])  # prevent flooding output

        print("\n💬 Inline Feedback (Instructor Comments):\n")
        print(example.get_comments_text() or "[No comments found]")

        print("\n📋 Rubric Feedback:\n")
        print(example.get_rubric_feedback_prompt() or "[No rubric feedback found]")

        print("\n" + "-"*80)


In [11]:
#from tropos import explore_examples

explore_examples(
    requirements_path="/content/project/data/Requirements.docx",
    example_dir="/content/project/data/raw"
)



📌 Assignment Requirements:

Real World Project
One of the things you'll find about being in college is how important questions are, especially big questions that don't have easy answers. The questions you choose from for this project are probably all being discussed in one course or another, but they are also big issues being discussed outside of academia.
For the project, choose one of the following to work with:
What is the importance of critical race theory?
What purpose does a mask mandate have?
How much do we need to be concerned about climate change?
What is behind the emphasis on STEM education?
Your overall goal for the project is to be able to provide the best answer you can for the question. You'll do this in three parts:
A Background that explains what the subject is and, briefly, what is being said about it. You and your readers need to know what it is and what it's about before you start discussing views on it. Key here is using objective sources for information.
Your Ana